<a href="https://colab.research.google.com/github/flash-berry/NLP/blob/main/%D0%B4%D0%B72_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Изменения:

1) Добавление токанайзера - Byte Pair Encoding с использованием регламентации на русском языке:


2) Добавление LayerNorm перед функцией активации Relu в классе FeedForward

Итог:
  - loss упал с 2.1231 до 1.9626

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import zipfile
import os
import regex as re
import copy
from itertools import chain

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

zip_path = '/content/texts.zip'

extract_dir = 'extract_dir'
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extract_dir)

txt_files = [f for f in os.listdir(extract_dir) if f.endswith('.txt')]

text = ''
for file in txt_files:
  file_path = os.path.join(extract_dir, file)
  with open(file_path, 'r', encoding='utf-8') as f:
    text += f.read()

#-------------------------------------------------------------------------------------------
# Встраиваем токенайзер BPE c регулярными выражениями для русского языка
# ------------------------------------------------------------------------------------------
advanced_russian_pat = re.compile(
    r"""
    # Английские клитики (можно удалить, если работаем только с русским)
    's|'t|'re|'ve|'m|'ll|'d|

    # Русские слова с апострофами и дефисами ("кой-какой", "чудо-юдо")
    (?:[ёЁа-яА-Я]+(?:['-][ёЁа-яА-Я]+)*)|

    # Сокращения ("т.д.", "т.п.", "и.о.")
    (?:[ёЁа-яА-Я]\.){1,3}|

    # Числа (включая числа с разделителями: 1_000_000, 1,234)
    \d+(?:[_,]\d+)*|

    # Остальные последовательности символов (знаки препинания, символы и т.д.)
    [^\sёЁа-яА-Я\d]+|

    # Пробелы (но не перед другим пробелом)
    \s+(?!\S)|

    # Любые оставшиеся пробелы
    \s+
    """,
    re.VERBOSE | re.IGNORECASE
)

tokens = re.findall(advanced_russian_pat, text)

tton = [] #tokens to numbers size: (number tokens, size tokens)
for i in range(len(tokens)):
  tton.append(list(map(int, tokens[i].encode("utf-8"))))

def get_stats(tton):
    counts = {}
    for i in range(len(tton)):
      for j in range(len(tton[i])-1):
          pair = (tton[i][j], tton[i][j+1])
          counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(tton, pair, idx):
  newids = []

  for i in range(len(tton)):
    newids.append([])
    j = 0
    while j < len(tton[i]):
      if j < len(tton[i]) - 1 and tton[i][j] == pair[0] and tton[i][j+1] == pair[1]:
        newids[i].append(idx)
        j += 2
      else:
        newids[i].append(tton[i][j])
        j += 1
  return newids

vocab_size = 276
num_merge = vocab_size - 256
ids = copy.deepcopy(tton)

merges = {}
for i in range(num_merge):
  stats = get_stats(ids)
  pair = max(stats, key = stats.get)
  idx = 256 + i
  print(f'merging {pair} into a new token {idx}')
  ids = merge(ids, pair, idx)
  merges[pair] = idx

vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
  vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors = "replace")
  return text

def encode(text):
    tokens = re.findall(advanced_russian_pat, text)
    tton = [] #tokens to numbers size: (number tokens, size tokens)
    for i in range(len(tokens)):
      tton.append(list(map(int, tokens[i].encode("utf-8"))))

    while True:
        pairs = get_stats(tton)
        if not pairs:
            break
        pair = None
        for p in pairs:
            if p in merges:
                pair = p
                break
        if not pair:
            break
        idx = merges[pair]
        tton = merge(tton, pair, idx)

    return list(chain.from_iterable(tton))

text2 = decode(encode(text))
print(text2 == text)
# ------------------------------------------------------------------------------------------

## here are all the unique characters that occur in this text
#chars = sorted(list(set(text)))
#vocab_size = len(chars)
## create a mapping from characters to integers
#stoi = { ch:i for i,ch in enumerate(chars) }
#itos = { i:ch for i,ch in enumerate(chars) }
#encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
#decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.LayerNorm(4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


merging (208, 190) into a new token 256
merging (208, 181) into a new token 257
merging (208, 176) into a new token 258
merging (208, 189) into a new token 259
merging (208, 184) into a new token 260
merging (209, 130) into a new token 261
merging (256, 208) into a new token 262
merging (209, 129) into a new token 263
merging (209, 128) into a new token 264
merging (208, 187) into a new token 265
merging (208, 178) into a new token 266
merging (209, 131) into a new token 267
merging (208, 180) into a new token 268
merging (208, 188) into a new token 269
merging (209, 139) into a new token 270
merging (208, 186) into a new token 271
merging (209, 140) into a new token 272
merging (208, 191) into a new token 273
merging (209, 143) into a new token 274
merging (208, 177) into a new token 275
True
0.238996 M parameters
step 0: train loss 5.7586, val loss 5.7644
step 100: train loss 2.7742, val loss 2.8776
step 200: train loss 2.5462, val loss 2.5752
step 300: train loss 2.4638, val loss 2.